In [15]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from datetime import date

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import yaml, pdb
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from email_validator import validate_email, EmailNotValidError
import config as cfg
import time
import random
import time, random, csv, pyautogui, pdb, traceback, sys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from datetime import date
from itertools import product
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options



ModuleNotFoundError: No module named 'config'

In [13]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
from webdriver_manager.chrome import ChromeDriverManager



In [8]:

def login_to_linkedin(driver):
    driver.get(constants.LINKEDIN_LOGIN_URL)
    username = driver.find_element(By.ID, 'username')
    password = driver.find_element(By.ID, 'password')
    username.send_keys(config.LINKEDIN_USERNAME)
    password.send_keys(config.LINKEDIN_PASSWORD)
    driver.find_element(By.XPATH, '//*[@type="submit"]').click()

def search_jobs(driver):
    driver.get(constants.LINKEDIN_JOBS_SEARCH_URL)
    search_keyword = driver.find_element(By.XPATH, '//*[@aria-label="Search by title, skill, or company"]')
    search_keyword.send_keys(config.JOB_SEARCH_KEYWORDS[0])  # For simplicity, using the first keyword
    search_location = driver.find_element(By.XPATH, '//*[@aria-label="City, state, or zip code"]')
    search_location.clear()
    search_location.send_keys(config.JOB_LOCATION)
    search_location.send_keys(Keys.RETURN)
    time.sleep(3)  # Wait for the page to load results

def apply_to_jobs(driver):
    jobs = driver.find_elements(By.CLASS_NAME, 'job-card-container')
    for job in jobs:
        job.click()
        time.sleep(2)  # Wait for the job details to load
        try:
            easy_apply_button = driver.find_element(By.CLASS_NAME, constants.EASY_APPLY_BUTTON_CLASS)
            easy_apply_button.click()
            time.sleep(2)  # Wait for the Easy Apply form to load
            handle_questions(driver)
            submit_application(driver)
        except Exception as e:
            print(f"Could not apply to job: {e}")
            continue


In [9]:
# This handles questions. You can add your codes to handle more questions.
# TODO: This function is broken because LinkedIn changed their front-end. 
# All the find_element() functions should be fixed according to current LinkedIn's front-end.
def handle_questions(questions):
    seen = set()
    
    # Work on the question asking about experience
    for question in questions:
        if question in seen: continue
            
        try:
            question_label = question.find_element(By.CSS_SELECTOR, "label")
            input_field = question.find_element(By.CSS_SELECTOR, "input[type='text']")
        except:
            continue

        question_text = question_label.text.lower()
        is_required = input_field.get_attribute("required")
        
        if is_required == "false":
            continue  # Skip if the question is not required

        # Check if the question asks for years of experience and if the input field is empty, then fill it with '1'
        if 'how many year' in question_text:
            # If input field is empty, fill it with '1'
            if not input_field.get_attribute('value'):
                input_field.send_keys('1')
                seen.add(question)
                      
        elif "city" in question_text:
            # Check if the text input is empty
            current_value = input_field.get_attribute("value")
            if not current_value:  # If the input field is empty
                input_field.send_keys("Los Angeles")
                time.sleep(random.uniform(1, constants.botSpeed))

                # Press the down arrow and then enter to select the first item
                input_field.send_keys(Keys.ARROW_DOWN, Keys.RETURN)
                seen.add(question)
                

    # Work on the question asking about a Bachelor's Degree
    for question in questions:
        if question in seen: continue
        # Try to find the legend and radio buttons within the current question
        try:
            legend = question.find_element(By.CSS_SELECTOR, "legend span.fb-dash-form-element__label")
            radio_yes = question.find_element(By.CSS_SELECTOR, "input[data-test-text-selectable-option__input='Yes']")
            label_yes = question.find_element(By.CSS_SELECTOR, "label[data-test-text-selectable-option__label='Yes']")
        except:
            # If legend or radio buttons not found, skip to next question
            continue

        # Check if the question is required
        is_required = radio_yes.get_attribute("aria-required")
        if is_required == "false":
            continue  # Skip if the question is not required

        # Check if the legend text is asking about Bachelor's Degree and select 'Yes'
        if "degree" in legend.text.lower() or "master" in legend.text.lower() or "bachelor" in legend.text.lower():
            label_yes.click()
            seen.add(question)
    
    
    # Work on the questions asking "Are you able?" -> "Yes!"
    for question in questions:
        if question in seen: continue
        # Try to find the label for dropdowns
        try:
            dropdown_label = question.find_element(By.CSS_SELECTOR, "label.fb-dash-form-element__label")
            dropdown = question.find_element(By.CSS_SELECTOR, "select[data-test-text-entity-list-form-select]")
        except:
            # If label or dropdown not found, skip to next question
            continue

        # Check if the question is required
        is_required = dropdown.get_attribute("aria-required")
        if is_required == "false":
            continue  # Skip if the question is not required

        # Check if the label text contains 'are you able' and select 'Yes'
        if "eligible to work" in dropdown_label.text.lower() or "job location" in dropdown_label.text.lower() or "responsible" in dropdown_label.text.lower() or "able" in dropdown_label.text.lower() or "willing" in dropdown_label.text.lower() or "comfortable" in dropdown_label.text.lower() or "open" in dropdown_label.text.lower() or "relocating" in dropdown_label.text.lower() or "passion" in dropdown_label.text.lower():
            select = Select(dropdown)
            select.select_by_visible_text("Yes")
            seen.add(question)
        
        # which major?
        elif "degree" in dropdown_label.text.lower():
            if "computer" in dropdown_label.text.lower():
                select = Select(dropdown)
                select.select_by_visible_text("Yes")
                seen.add(question)
            else:
                select = Select(dropdown)
                select.select_by_visible_text("No")
                seen.add(question)
            
                
    # disability status and how did you hear
    for question in questions:
        if question in seen: continue
        # Check if it's a dropdown for the Disability Status
        try:
            dropdown_label = question.find_element(By.CSS_SELECTOR, "label")
            dropdown = question.find_element(By.CSS_SELECTOR, "select")
        except:
            # If label or dropdown not found, skip to next question
            continue

        # Check if the label text contains 'Disability Status'
        if "disability" in dropdown_label.text.lower():
            select = Select(dropdown)
            # Iterate through options and select the one starting with 'No'
            for option in select.options:
                if option.text.lower().startswith("no"):
                    select.select_by_visible_text(option.text)
                    seen.add(question)
                    break
                    
        elif "how" in dropdown_label.text.lower() and "hear" in dropdown_label.text.lower() and "about" in dropdown_label.text.lower():
            select = Select(dropdown)
            # Iterate through options and select the one starting with 'No'
            for option in select.options:
                if option.text.lower().startswith("linkedin"):
                    select.select_by_visible_text(option.text)
                    seen.add(question)
                    break
                    
        elif "veteran" in dropdown_label.text.lower():
            select = Select(dropdown)
            # Iterate through options and select the one starting with 'No'
            for option in select.options:
                if option.text.lower().startswith("i am not") or option.text.lower().startswith("no"):
                    select.select_by_visible_text(option.text)
                    seen.add(question)
                    break
                

    # Race
    for question in questions:
        if question in seen: continue
        # Find the dropdown
        try:
            dropdown = question.find_element(By.CSS_SELECTOR, "select")
        except:
            # If dropdown not found, skip to next question
            continue

        select = Select(dropdown)
        
        for option in select.options:
            if 'asian' in option.text.lower():
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
                
            elif option.text.lower().startswith("male"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
            
            elif option.text.lower().startswith("man"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break    
                
            elif option.text.lower().startswith("he/him"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
                
            elif option.text.lower().startswith("he, him"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
                
            elif option.text.lower().startswith("i agree"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break    
                
            elif option.text.lower().startswith("i acknowledge"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
            

    
#     # legal name
#     for question in questions:
#         if question in seen: continue
#         # Find the input for Legal Name (if exists)
#         try:
#             input_element = question.find_element(By.XPATH, '//input[@type="text"]')
#             label = question.find_element(By.XPATH, '//label')
#         except:
#             # If input not found, skip to next question
#             continue

#         # Check if the label is for Legal Name
#         if "legal name" in label.text.lower() or "your name" in label.text.lower():
#             # Input "Samuel Shin"
#             if not input_element.get_attribute('value'):
#                 input_element.send_keys("Samuel Shin")
#                 seen.add(question)
        
#         elif "first name" in label.text.lower():
#             # Input "Samuel Shin"
#             if not input_element.get_attribute('value'):
#                 input_element.send_keys("Samuel")
#                 seen.add(question)
            
            
    # hear about this job and address
    for question in questions:
        if question in seen: continue
        # Try to find a label within the question
        labels = question.find_elements(By.CSS_SELECTOR, 'label')

        for label in labels:
            # If the label text contains "how did you hear about this job"
            if "how" in label.text.lower() and "hear" in label.text.lower() and "about" in label.text.lower():

                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    input_element.send_keys("LinkedIn")
                    seen.add(question)
                    
            # Check if the label is for Legal Name
            elif "legal name" in label.text.lower() or "your name" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    input_element.send_keys("Sangil Jung")
                    seen.add(question)
                    
            # Check if the label is for Legal Name
            elif "first name" in label.text.lower() or "preferred name" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    input_element.send_keys("Sangil")
                    seen.add(question)
                    
                
    # Visa sponsorship
    for question in questions:
        if question in seen: continue
        # Try to find a label within the question
        labels = question.find_elements(By.CSS_SELECTOR, 'label')

        for label in labels:
            # If the label text contains "require immigration sponsorship"
            if "do you now or will you" in label.text.lower() or "sponsorship" in label.text.lower() or "in the future" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the select element
                select_id = label.get_attribute('for')

                # Locate the corresponding select element using the extracted id
                select_element = question.find_element(By.ID, select_id)

                # Use Select class to interact with the <select> element
                select = Select(select_element)

                # Select "No" by its visible text
                select.select_by_visible_text("No")
                seen.add(question)

        
    # LinkedIn and Salary
    for question in questions:
        if question in seen: continue
        # Try to find a label within the question
        labels = question.find_elements(By.CSS_SELECTOR, 'label')

        for label in labels:
            # If the label text contains "include your LinkedIn profile"
            if "linkedin" in label.text.lower():

                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Enter the LinkedIn profile URL
                if not input_element.get_attribute('value'):
                    input_element.send_keys("https://www.linkedin.com/in/sangiljung/")
                    seen.add(question)
                
            # If the label text contains "include your LinkedIn profile"
            elif "portfolio" in label.text.lower():

                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Enter the LinkedIn profile URL
                if not input_element.get_attribute('value'):
                    input_element.send_keys("https://sangiljung.oopy.io/")
                    seen.add(question)
                
            elif "expected" in label.text.lower() or "compensation" in label.text.lower() or "salary" in label.text.lower() or "usd" in label.text.lower() or "expectation" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Enter the LinkedIn profile URL
                if not input_element.get_attribute('value'):
                    input_element.send_keys("120000")
                    seen.add(question)
                
            elif "address" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    if "email" in label.text.lower():
                        input_element.send_keys("sangilish@gmail.com")
                        seen.add(question)
                    else:
                        input_element.send_keys("525 Del Rey Dr, Placentia, CA")
                        seen.add(question)
                        
            elif "state" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    input_element.send_keys("California")
                    seen.add(question)
                    
            elif "zip code" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    input_element.send_keys("92870")
                    seen.add(question)
                    
            elif "relationship" in label.text.lower() or "government" in label.text.lower():
                # Extract the 'for' attribute which contains the id of the input element
                input_id = label.get_attribute('for')

                # Locate the corresponding input element using the extracted id
                input_element = question.find_element(By.ID, input_id)

                # Input "LinkedIn"
                if not input_element.get_attribute('value'):
                    input_element.send_keys("N/A")
                    seen.add(question)
    
    # have you been employed and clearance
    for question in questions:
        if question in seen: continue
        # Try to find a label within the question
        labels = question.find_elements(By.CSS_SELECTOR, 'label')

        for label in labels:
            # If the label text contains "Have you been previously employed by"
            if "currently work for" in label.text.lower() or "employee" in label.text.lower() or "employed" in label.text.lower() or "ts" in label.text.lower() or "sci" in label.text.lower() or 'veteran' in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'No' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("no"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
            elif "previously" in label.text.lower() and ("applied" in label.text.lower() or "worked" in label.text.lower()):
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'No' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("no"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
                    
            elif "have" in label.text.lower() and ("license" in label.text.lower() or "certification" in label.text.lower()):
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'No' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("no"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
                    
                                    
            elif "eligible to work" in label.text.lower() or "willing" in label.text.lower() or "relocate" in label.text.lower() or "relocating" in label.text.lower() or "comfortable" in label.text.lower() or "commuting" in label.text.lower() or "commute" in label.text.lower() or "18 years" in label.text.lower() or "older" in label.text.lower() or "authorized" in label.text.lower() or "or will you" in label.text.lower() or "relocation" in label.text.lower() or "open" in label.text.lower() or "passion" in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'Yes' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("yes"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
                    
            elif ("reside" in label.text.lower() or "live" in label.text.lower()) and ("u.s" in label.text.lower() or "states" in label.text.lower() or "us" in label.text.lower()):
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'Yes' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("yes"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
                    
            # degree completion
            elif "complet" in label.text.lower() and ("master" in label.text.lower() or "bachelor" in label.text.lower() or "degree" in label.text.lower() or "high school" in label.text.lower() or "diploma" in label.text.lower()):
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'Yes' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("yes"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
                    
            # currently live
            elif "currently" in label.text.lower() and "live" in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'Yes' by visible text
                    if "la" in label.text.lower() or "ca" in label.text.lower() or "los angeles" in label.text.lower() or "california" in label.text.lower() or "states" in label.text.lower():
                        for option in select.options:
                            if option.text.lower().startswith("yes"):
                                select.select_by_visible_text(option.text)
                                seen.add(question)
                                break
                    else:
                        for option in select.options:
                            if option.text.lower().startswith("no"):
                                select.select_by_visible_text(option.text)
                                seen.add(question)
                                break
                except:
                    continue
                    
            

                            
    # professional experience and work authorization
    for question in questions:
        if question in seen: continue
        # Try to find a label within the question
        labels = question.find_elements(By.CSS_SELECTOR, 'label')

        for label in labels:
            if "compensation" in label.text.lower():
            
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select 'No' by visible text
                    for option in select.options:
                        if option.text.lower().startswith("$12"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                        elif option.text.lower().startswith("$10"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue

            elif "work authorization" in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    # Create a Select object
                    select = Select(select_element)

                    # Select the option by visible text
                    for option in select.options:
                        if option.text.lower().startswith("i am authorized"):
                            select.select_by_visible_text(option.text)
                            seen.add(question)
                            break
                except:
                    continue
                    
            elif "background" in label.text.lower() or "experience" in label.text.lower() or "authorized" in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    if select_element:
                        # Create a Select object
                        select = Select(select_element)

                        # Select the option "Yes" by visible text
                        for option in select.options:
                            if option.text.lower().startswith("yes"):
                                select.select_by_visible_text(option.text)
                                seen.add(question)
                                break
                except:
                    continue
                
            # Check if it's a text input question
            elif "years" in label.text.lower() and "experience" in label.text.lower():
                try:
                    # Locate the corresponding input element
                    input_element = question.find_element(By.CSS_SELECTOR, 'input[type="text"]')

                    if input_element:
                        # Input "Yes" into the text field
                        if not input_element.get_attribute('value'):
                            input_element.send_keys("Yes")
                            seen.add(question)
                except:
                    continue
            
            
            # Check if it's a dropdown selection question for TS/SCI with full-scope poly
            elif "veteran" in label.text.lower() or "ts/sci" in label.text.lower() or "clearance" in label.text.lower() or 'noaa' in label.text.lower() or 'usphs' in label.text.lower() or 'armed' in label.text.lower() or 'forces' in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    if select_element:
                        # Create a Select object
                        select = Select(select_element)

                        # Select the option "No" by visible text
                        for option in select.options:
                            if option.text.lower().startswith("no"):
                                select.select_by_visible_text(option.text)
                                seen.add(question)
                                break
                except:
                    continue
                
            # based on
            elif "based" in label.text.lower() and "legally" not in label.text.lower() and "authorized" not in label.text.lower():
                try:
                    # Locate the corresponding select element
                    select_element = question.find_element(By.CSS_SELECTOR, 'select')

                    if select_element:
                        # Create a Select object
                        select = Select(select_element)
                        if "los angeles" in label.text.lower() or "california" in label.text.lower():
                            # Select the option "No" by visible text
                            for option in select.options:
                                if option.text.lower().startswith("yes"):
                                    select.select_by_visible_text(option.text)
                                    seen.add(question)
                                    break
                        else:
                            for option in select.options:
                                if option.text.lower().startswith("no"):
                                    select.select_by_visible_text(option.text)
                                    seen.add(question)
                                    break
                except:
                    continue
                
            # comfortable with commuting
            elif "comfortable" in label.text.lower() or "commuting" in label.text.lower() or "commute" in label.text.lower():
                try:
                    label_yes = question.find_element(By.XPATH, '//label[contains(text(), "Yes")]')
                    if label_yes:
                        # Click the label for the "Yes" radio button
                        label_yes.click()
                        seen.add(question)
                except:
                    continue
                    
            elif "start" in label.text.lower() and "immediately" in label.text.lower():
                try:
                    label_yes = question.find_element(By.XPATH, '//label[contains(text(), "Yes")]')
                    if label_yes:
                        # Click the label for the "Yes" radio button
                        label_yes.click()
                        seen.add(question)
                except:
                    continue
                    
                        # degree completion
            elif "complet" in label.text.lower() and ("master" in label.text.lower() or "bachelor" in label.text.lower() or "degree" in label.text.lower() or "high school" in label.text.lower() or "diploma" in label.text.lower()):
                try:
                    label_yes = question.find_element(By.XPATH, '//label[contains(text(), "Yes")]')
                    if label_yes:
                        # Click the label for the "Yes" radio button
                        label_yes.click()
                        seen.add(question)
                except:
                    continue

                
    for question in questions:
        if question in seen: continue
        # Try to find a legend within the question
        legends = question.find_elements(By.CSS_SELECTOR, 'legend')

        for legend in legends:
            # Privacy policy
            if "privacy" in legend.text.lower() or "understand" in legend.text.lower() or "knowledge" in legend.text.lower():
                try:
                    # Locate the corresponding label element
                    label = question.find_element(By.CSS_SELECTOR, 'label[for^="urn:li:fsd_formElement:"]')

                    if label:
                        # Click the label to check the checkbox
                        label.click()
                        seen.add(question)
                except:
                    continue
            
            # Check if it's a question about commuting
            elif "eligible to work" in legend.text.lower() or "willing" in legend.text.lower() or "relocate" in legend.text.lower() or "relocating" in legend.text.lower() or "comfortable" in legend.text.lower() or "commuting" in legend.text.lower() or "18 years" in legend.text.lower() or "older" in legend.text.lower() or "authorized" in legend.text.lower() or "or will you" in legend.text.lower() or "relocation" in legend.text.lower() or "open" in legend.text.lower():
                try:
                    # Locate the label for the "Yes" radio button inside the question
                    label_yes = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Yes")]')

                    if label_yes:
                        # Click the label for the "Yes" radio button
                        label_yes.click()
                        seen.add(question)
                except:
                    continue
                    
            elif ("reside" in legend.text.lower() or "live" in legend.text.lower()) and ("u.s" in legend.text.lower() or "states" in legend.text.lower() or "us" in legend.text.lower()):
                try:
                    # Locate the label for the "Yes" radio button inside the question
                    label_yes = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Yes")]')

                    if label_yes:
                        # Click the label for the "Yes" radio button
                        label_yes.click()
                        seen.add(question)
                except:
                    continue
                    
            elif "start" in label.text.lower() and "immediately" in label.text.lower():
                try:
                    # Locate the label for the "Yes" radio button inside the question
                    label_yes = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Yes")]')

                    if label_yes:
                        # Click the label for the "Yes" radio button
                        label_yes.click()
                        seen.add(question)
                except:
                    continue
                    
            elif "hispanic" in legend.text.lower() or "latino" in legend.text.lower() or "veteran" in legend.text.lower() or "sponsorship" in legend.text.lower() or "employed" in legend.text.lower():
                try:
                    # Locate the label for the "Yes" radio button inside the question
                    label_no = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "No")]')

                    if label_no:
                        label_no.click()
                        seen.add(question)
                except:
                    continue
                    
            elif ("previous" in legend.text.lower() or "previously" in legend.text.lower()) and "employee" in legend.text.lower():
                try:
                    # Locate the label for the "Yes" radio button inside the question
                    label_no = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "No")]')

                    if label_no:
                        label_no.click()
                        seen.add(question)
                except:
                    continue
                    
                    
            elif "noaa" in legend.text.lower() or "usphs" in legend.text.lower() or "armed force" in legend.text.lower() or "commissioned corps" in legend.text.lower():
                try:
                    # Locate the label for the "Yes" radio button inside the question
                    label_no = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "No")]')

                    if label_no:
                        # Click the label for the "Yes" radio button
                        label_no.click()
                        seen.add(question)
                except:
                    continue
                    
            # Check if it's a question about disability
            elif "disability" in legend.text.lower() or "transgender" in legend.text.lower():
                try:
                    # Locate the label for the "No" radio button inside the question
                    label_no = question.find_element(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "No")]')

                    if label_no:
                        # Click the label for the "No" radio button
                        label_no.click()
                        seen.add(question)
                except:
                    continue
                    

            # Check if it's a question about disability
            elif "gender" in legend.text.lower():
#                 try:
                # Locate the label for the "No" radio button inside the question
                label_man = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="Man"]')

                if label_man:
                    # Click the label for the "No" radio button
                    label_man.click()
                    seen.add(question)

                # Locate the label for the "No" radio button inside the question
                label_male = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="Male"]')

                if label_male:
                    # Click the label for the "No" radio button
                    label_male.click()
                    seen.add(question)                    
#                 except:
#                     continue
                    
            # Check if it's a question about disability
            elif "race" in legend.text.lower() or "racial" in legend.text.lower() or "ethnic" in legend.text.lower() or "ethnicity" in legend.text.lower():
                try:
                    # Locate the label for the "No" radio button inside the question
                    label_race1 = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="East Asian"]')
                                                        
                    if label_race1:
                        # Click the label for the "No" radio button
                        label_race1.click()
                        seen.add(question)
                        
                    # Locate the label for the "No" radio button inside the question
                    label_race2 = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="Asian"]')

                    if label_race2:
                        # Click the label for the "No" radio button
                        label_race2.click()
                        seen.add(question)                    
                except:
                    continue
                    
            elif "sexual" in legend.text.lower() or "sex" in legend.text.lower():
                try:
                    # Locate the label for the "No" radio button inside the question
                    label_race1 = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="Heterosexual"]')

                    if label_race1:
                        # Click the label for the "No" radio button
                        label_race1.click()
                        seen.add(question)
                        
                    # Locate the label for the "No" radio button inside the question
                    label_race2 = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="Straight"]')

                    if label_race2:
                        # Click the label for the "No" radio button
                        label_race2.click()
                        seen.add(question)                    
                except:
                    continue
                    
            elif "have" in legend.text.lower() and ("license" in legend.text.lower() or "certification" in legend.text.lower()):
                try:
                    # Locate the label for the "No" radio button inside the question
                    label_license = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="No"]')

                    if label_license:
                        # Click the label for the "No" radio button
                        label_license.click()
                        seen.add(question)         
                except:
                    continue
                    
            elif "complet" in legend.text.lower() and ("master" in legend.text.lower() or "bachelor" in legend.text.lower() or "degree" in legend.text.lower() or "high school" in legend.text.lower() or "diploma" in legend.text.lower()):
                try:
                    # Locate the label for the "No" radio button inside the question
                    label_license = question.find_element(By.XPATH, './/label[@data-test-text-selectable-option__label="Yes"]')

                    if label_license:
                        # Click the label for the "No" radio button
                        label_license.click()
                        seen.add(question)         
                except:
                    continue
        
    for question in questions:
        if question in seen: continue
        
        # Locate the label for the "Yes" radio button inside the question
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "I am not a protected veteran")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue
            
        # Locate the label for the "Yes" radio button inside the question
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "No, I Don\'t Have A Disability, Or A History/Record Of Having A Disability")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue

        # Locate the label for the "Yes" radio button inside the question
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Male")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue
            
        # Locate the label for the "Yes" radio button inside the question
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Non-citizen allowed to work for any employer")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue
        
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Asian")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue
            
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "East Asian")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue
            
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Not Hispanic or Latino")]')
        if label_yes:
            # Click the label for the "Yes" radio button
            label_yes[0].click()
            seen.add(question)
            continue
            
            
    time.sleep(random.uniform(1, constants.botSpeed))

In [14]:
def submit_application(driver):
    try:
        submit_button = driver.find_element(By.CLASS_NAME, constants.SUBMIT_APPLICATION_BUTTON_CLASS)
        submit_button.click()
        time.sleep(2)  # Wait for the application to be submitted
    except Exception as e:
        print(f"Could not submit application: {e}")

def main():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    login_to_linkedin(driver)
    search_jobs(driver)
    apply_to_jobs(driver)
    driver.quit()

if __name__ == '__main__':
    start = time.time()
    main()
    end = time.time()
    print(f"--- Took: {round((end - start) / 60, 2)} minute(s).")

AttributeError: 'str' object has no attribute 'capabilities'

In [25]:

start = time.time()
Linkedin().linkJobApply()
end = time.time()
utils.prYellow("---Took: " + str(round((time.time() - start)/60)) + " minute(s).")

🤖 Thanks for using Easy Apply Jobs bot, for more information you can visit our site - www.automated-bots.com
🌐 Bot will run in Chrome browser and log in Linkedin for you.


KeyboardInterrupt: 